In [1]:
# import required modules for this project

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load csv files
attendance = pd.read_csv('/Users/sa26/Documents/GitHub/Predict-Podcast-Listening-Time/Education-Capstone/data/2016-17_-_2020-23_Citywide_End-of-Year_Attendance_and_Chronic_Absenteeism_Data_20250604.csv')
demographics = pd.read_csv('/Users/sa26/Documents/GitHub/Predict-Podcast-Listening-Time/Education-Capstone/data/2019-20_Demographic_Snapshot_-_Citywide_20250604.csv')
graduation = pd.read_csv('/Users/sa26/Documents/GitHub/Predict-Podcast-Listening-Time/Education-Capstone/data/Graduation_results_for_Cohorts_2012_to_2019__Classes_of_2016_to_2023__20250609.csv')